## EAoU-4 Master Notebook
ΛCDM vs EAoU fits on GRBs, Pantheon+SH0ES, Quasars, and Combined.

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import minimize, minimize_scalar

plt.rcParams["figure.figsize"] = (8,5)
plt.rcParams["axes.grid"] = True
c_over_H0 = 2997.92458  # Mpc (c/H0 with H0=100 km/s/Mpc)

# Cosmology functions
def Ez(z, Om):
    Ol = 1.0 - Om
    return np.sqrt(Om*(1+z)**3 + Ol)

def chi_comoving(z, Om):
    val, _ = quad(lambda zp: 1.0/Ez(zp, Om), 0.0, z, limit=200)
    return val

def mu_from_dL(dL):
    return 5.0*np.log10(dL) + 25.0

def mu_model_LCDM(z, Om):
    z = np.atleast_1d(z)
    dL = np.array([(1+zz)*chi_comoving(zz, Om) for zz in z]) * c_over_H0
    return mu_from_dL(dL)

def mu_model_EAoU(z, Om, alpha):
    z = np.atleast_1d(z)
    dL = np.array([(1+zz)**(1.0+alpha)*chi_comoving(zz, Om) for zz in z]) * c_over_H0
    return mu_from_dL(dL)


In [ ]:

def chi2_marginalized(mu_obs, mu_th, sigma):
    w   = 1.0/np.square(sigma)
    A   = np.sum(w)
    B   = np.sum(w*(mu_obs - mu_th))
    C   = np.sum(w*(mu_obs - mu_th)**2)
    chi2 = C - B*B/A
    dof = len(mu_obs) - 1
    return chi2, dof

def fit_LCDM(z, mu, sigma, Om_guess=0.3):
    def f(Om):
        if not (0.02 <= Om <= 0.8): return np.inf
        mu_th = mu_model_LCDM(z, Om)
        chi2, _ = chi2_marginalized(mu, mu_th, sigma)
        return chi2
    res = minimize_scalar(f, bounds=(0.02, 0.8), method="bounded", options={"xatol":1e-3})
    Om_best = float(res.x)
    mu_th   = mu_model_LCDM(z, Om_best)
    chi2, dof = chi2_marginalized(mu, mu_th, sigma)
    return {"Om": Om_best, "alpha": np.nan, "chi2": chi2, "dof": dof, "chi2_red": chi2/dof}

def fit_EAoU(z, mu, sigma, Om_guess=0.3, alpha_guess=0.0):
    def f(p):
        Om, alpha = p
        if not (0.02 <= Om <= 0.8):    return np.inf
        if not (-0.5 <= alpha <= 0.5): return np.inf
        mu_th = mu_model_EAoU(z, Om, alpha)
        chi2, _ = chi2_marginalized(mu, mu_th, sigma)
        return chi2
    res = minimize(f, x0=[Om_guess, alpha_guess], method="L-BFGS-B",
                   bounds=((0.02,0.8), (-0.5,0.5)))
    Om_b, alpha_b = map(float, res.x)
    mu_th = mu_model_EAoU(z, Om_b, alpha_b)
    chi2, dof = chi2_marginalized(mu, mu_th, sigma)
    return {"Om": Om_b, "alpha": alpha_b, "chi2": chi2, "dof": dof, "chi2_red": chi2/dof}


In [ ]:

def summarize_block(name, z, mu, sigma, Om_guess=0.3, alpha_guess=0.0):
    lcdm = fit_LCDM(z, mu, sigma, Om_guess=Om_guess)
    eaou = fit_EAoU(z, mu, sigma, Om_guess=Om_guess, alpha_guess=alpha_guess)
    dchi2 = eaou["chi2"] - lcdm["chi2"]
    print(f"\nLoaded {name}: ({len(z)} objects)")
    print(f"{'Model':<6} {'Ωm':>8} {'α':>8} {'χ²':>10} {'dof':>6} {'χ²/ν':>10} {'Δχ² (EAoU−ΛCDM)':>18}")
    print(f"{'ΛCDM':<6} {lcdm['Om']:>8.5f} {np.nan:>8} {lcdm['chi2']:>10.5f} {lcdm['dof']:>6} {lcdm['chi2_red']:>10.5f} {np.nan:>18}")
    print(f"{'EAoU':<6} {eaou['Om']:>8.5f} {eaou['alpha']:>8.5f} {eaou['chi2']:>10.5f} {eaou['dof']:>6} {eaou['chi2_red']:>10.5f} {dchi2:>18.5f}")
    return lcdm, eaou, dchi2

def plot_z_hist(title, z, bins=18):
    plt.figure()
    plt.hist(z, bins=bins, color="salmon", edgecolor="k")
    plt.title(f"{title} redshift distribution (N={len(z)})")
    plt.xlabel("z"); plt.ylabel("Count"); plt.show()


In [ ]:

fn_grb     = "GRB_All.csv"
fn_panth   = "Pantheon_SH0ES.csv"
fn_quasars = "quasars_A150_table3_full.csv"

def load_simple_csv(path, id_col="ID", z_col="z", mu_col="mu", s_col="sigma_mu", src_col="source"):
    df = pd.read_csv(path)
    rename_map = {}
    for want in [id_col, z_col, mu_col, s_col, src_col]:
        if want not in df.columns:
            for alt in [want, want.upper(), want.capitalize()]:
                if alt in df.columns: rename_map[alt] = want
    if rename_map:
        df = df.rename(columns=rename_map)
    need = {id_col, z_col, mu_col, s_col, src_col}
    missing = list(need - set(df.columns))
    if missing:
        raise ValueError(f"{path} is missing columns: {missing}")
    df = df[[id_col, z_col, mu_col, s_col, src_col]].dropna()
    df = df[(df[z_col] > 0) & (df[s_col] > 0)]
    z  = df[z_col].to_numpy()
    mu = df[mu_col].to_numpy()
    su = df[s_col].to_numpy()
    return z, mu, su, df

z_grb, mu_grb, su_grb, df_grb = load_simple_csv(fn_grb)
z_sn,  mu_sn,  su_sn,  df_sn  = load_simple_csv(fn_panth)
z_q,   mu_q,   su_q,   df_q   = load_simple_csv(fn_quasars)

len(z_grb), len(z_sn), len(z_q)


In [ ]:

lcdm_g, eaou_g, dchi2_g = summarize_block("GRBs", z_grb, mu_grb, su_grb, Om_guess=0.30, alpha_guess=0.00)
plot_z_hist("GRBs", z_grb)

lcdm_s, eaou_s, dchi2_s = summarize_block("Pantheon+SH0ES", z_sn, mu_sn, su_sn, Om_guess=0.30, alpha_guess=0.00)
plot_z_hist("Pantheon+SH0ES", z_sn)

lcdm_q, eaou_q, dchi2_q = summarize_block("Quasars", z_q, mu_q, su_q, Om_guess=0.30, alpha_guess=0.00)
plot_z_hist("Quasars", z_q)


In [ ]:

z_all  = np.concatenate([z_grb, z_sn, z_q])
mu_all = np.concatenate([mu_grb, mu_sn, mu_q])
su_all = np.concatenate([su_grb, su_sn, su_q])

lcdm_a, eaou_a, dchi2_a = summarize_block("Combined (GRB+Pantheon+Quasars)", z_all, mu_all, su_all,
                                          Om_guess=0.30, alpha_guess=0.00)
plot_z_hist("Combined", z_all, bins=30)
